# 第三周作业 井字棋胜负判断

---
**摘要**：
按照 Udacity ID3 Algorithm for Decision Trees (PDF) 中的介绍，用递归调用的方式，实现了一个基本的 ID3 决策树，用于根据井字棋的棋子布局对棋局胜负进行判断。

## 问题及数据集描述  
### 问题：  
    给出井字棋棋盘中所有空位的棋子布局，通过决策树实现对结果的预测  
  
### 数据集：  
    train.csv: 训练集 棋子布局，即 train_feature  
    y_train.csv: 训练集 比赛结果，即 train_label  
    test.csv: 测试集 棋子布局，即 test_feature  


---
## 算法原理

### 算法 ID3

讲述你的解决方案中以及探索过程中所用到算法及其原理。我们希望你在学习过程中就有记录笔记。可以整理后把对应放到此处。算法原理至少包含下述内容：

- 输入和输出
    - 输入：  
        - 训练决策树：def create_decision_tree(dataSet, featureNames) 
            - dataSet 即训练集
            - featureNames 即所有棋盘的空格
            
        - 预测结果：def predict(myDecisionTree, featureNames, test_dataset) 
            - test_dataset 即测试集棋子布局
    - 输出：  
        - 训练决策树：return myDecisionTree  
        - 预测结果：return myClassLabels  


- 原理（包含公式）  
    1. 信息熵：
    $Entropy(S) = \sum_{i=1}^{c}\log_2 p_i$
    2. 信息增益：
    $Gain(S, A) = Entropy(S) − \sum_{v\in Values(A)}\frac{|Sv|}{|S|}Entropy(S_v)$


- 算法所涉及的超参数所代表的含义、作用。尤其是在你的解决方案中涉及到的参数。对于这些超参数的值，你如何设定它们的值，改变这些值会对模型造成怎样的影响？
    - 由于尚未设置最大深度，故没有超参数的设定
    
    
- 算法的优点、缺点：
    - 优点：逻辑较容易理解
    - 缺点：容易在生成决策树的训练过程中出现过拟合的问题  


- 对于算法中你还不理解的地方，也应该写下来。在下周的例会中，我们会尽量做出解答

---
## 解决方案
- 正在尝试使用 skleran 中自带的 Class 如：LogisticRegression，KNeighborsClassifier，DecisionTreeClassifier 等进行训练，但需要将原 csv 文件中的 String 类型用 float 进行替换，尚未完成。

详细的描述你对井字棋分类问题的具体解决方案。我们也建议你记录下整个探索过程，包括没有用在最终解决方案中的模型。模型的执行过程，以及过程中遇到的困难的描述应该清晰明了地记录和描述。

要考虑下列问题：
- 你所用到的算法和技术执行的方式是否清晰记录了？
    是
- 在运用上面所提及的技术及指标的执行过程中是否遇到了困难，是否需要作出改动来得到想要的结果？
    是
- 是否有需要重点解释的代码片段(例如复杂的函数）？

## 实验结果

实验的结果，以及分析。

如果你尝试了多个模型，你可以用表格的形式呈现。

下图是示例，请根据自己的情况修改。

| 模型 | Local F1 score | Public Leaderboard |
| --- | --- | --- |
| ID3 决策树  |  | 0.80208 |
| XGBoost |  |  |
| Random Forest |  |  |
| Ensemble |  | -  |



## 总结

总结自己的工作，思考现有模型存在的问题，并指出可能的改进方法：   
- 模型尚未设置树的最大深度 max_depth 和每个叶节点的最小 samples 数，在训练过程中容易出现过拟合的情况。

## 代码

请把你的代码整理后放到下面。

In [1]:
# Step_1: 
# Import package

import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import sklearn

%matplotlib inline

In [2]:
# Step 2:
# Read data from train.csv and y_train.csv

train_feature = pd.read_csv('train.csv')
train_label = pd.read_csv('y_train.csv')

train_dataSet = pd.merge(train_feature, train_label, on = 'ID')

featureNames = train_dataSet.columns.tolist()[1:10]
featureNames_copy = train_dataSet.columns.tolist()[1:10]

dataSet_inArray = train_dataSet.values
dataSet_inArray_noSerial = dataSet_inArray[:,1:]
dataSet_totalSplit = dataSet_inArray_noSerial.tolist()

IOError: File train.csv does not exist

In [ ]:
# Step 3:
# Calculate the entropy given a dataset

def calculateShannonEntropy(dataSet): 
    numEntries = len(dataSet) # There are n rows inside
    labelCounts = {} # Create dictionary for classification

    for featureVector in dataSet:
        
    	currentLabel = featureVector[-1] # Get the last-row data
    	if currentLabel not in labelCounts.keys():
    		labelCounts[currentLabel] = 0
    	labelCounts[currentLabel] += 1

    total_entropy = 0.0
    for key in labelCounts:
    	proportion_k = float(labelCounts[key]) / numEntries
    	total_entropy -= (proportion_k * math.log(proportion_k, 2))

    return total_entropy

In [ ]:
# Step 3 Test:
calculateShannonEntropy(dataSet_totalSplit)

In [ ]:
# Step 4:
# Return the best feature based on the maximum number of information gain

def choose_best_feature_to_split(dataSet):    
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calculateShannonEntropy(dataSet)
    bestInfoGain = 0
    best_feature = -1

    for i in range(numFeatures):
    	featureList = [number[i] for number in dataSet] # enum for one attribute
    	uniqualValues = set(featureList) # no-relace attribute
    	newEntropy = 0

    	for value in uniqualValues:
    		sub_dataset = split_dataset(dataSet, i, value)
    		proportion_k = len(sub_dataset) / float(len(dataSet))
    		newEntropy += proportion_k * calculateShannonEntropy(sub_dataset) # sum(ShannonEntropy)
    	infoGain = baseEntropy - newEntropy # infoGain

    	# bestInfoGain
    	if (infoGain > bestInfoGain):
    		bestInfoGain = infoGain
    		best_feature = i

    return best_feature

In [ ]:
# Step 4 Test:
choose_best_feature_to_split(dataSet_totalSplit)

In [ ]:
# Step 5: 
# Split the dataset via current selected feature and it's value
# For example, when current_feature is TLS(top-left-square), and the value is 'o', 
# the task is that return the subdataset in which all "TLS" is equal to 'o'

def split_dataset(dataSet, axis, value):
    sub_dataset = []

    for featureVector in dataSet:
    	if featureVector[axis] == value:
    		reduceFeatureVector = featureVector[ :axis]
    		reduceFeatureVector.extend(featureVector[axis+1: ])  
    		sub_dataset.append(reduceFeatureVector)

    return sub_dataset

In [ ]:
# Step 6: 
# Create a decision tree by recursion

import operator
def majorityCnt(classList):
    classCount={}
    for vote in classList:
        if vote not in classCount.keys():classCount[vote] = 0
        classCount[vote]+=1
    sortedClassCount=sorted(classCount.items(),key = operator.itemgetter(1),reverse = True)
    return sortedClassCount[0][0]

def create_decision_tree(dataSet, featureNames):    
    classList = [example[-1] for example in dataSet]
    #类别相同，停止划分
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    #长度为1，返回出现次数最多的类别
    if len(classList[0]) == 1:
        return majorityCnt(classList)

    best_feature = choose_best_feature_to_split(dataSet) #返回分类的特征序号
    bestFeatureName = featureNames[best_feature] #该特征的label
    decision_tree = {bestFeatureName: { } }
    del(featureNames[best_feature]) #从labels的list中删除该label
    
    featureValues = [example[best_feature] for example in dataSet]
    uniqualValues = set(featureValues)
    for value in uniqualValues:
    	subFeatureNames = featureNames[ : ] #子集合

    	#构建数据的子集合，并进行递归
    	decision_tree[bestFeatureName][value] = create_decision_tree(split_dataset(dataSet, best_feature, value), subFeatureNames)
    
    return decision_tree

In [ ]:
# Step 5&6 Test:
myDecisionTree = create_decision_tree(dataSet_totalSplit, featureNames)
myDecisionTree

In [ ]:
# Step 7
# Func: classify

def classify(inputTree, featureNames, testVector):
    classLabel = []
    firstStr = inputTree.keys()[0] #获取树的第一个特征属性
    secondDict = inputTree[firstStr] #树的分支，子集合Dict
    featureIndex = featureNames.index(firstStr) #获取决策树第一层在featLables中的位置
    for key in secondDict.keys():
        if testVector[featureIndex] == key:
            if type(secondDict[key]).__name__ == 'dict':
                classLabel = classify(secondDict[key], featureNames, testVector)
            else:
            	classLabel = secondDict[key]
    
    return classLabel

In [ ]:
# Step 7 Test
# Func: classify Test:
classLabel = classify(myDecisionTree, featureNames_copy, ['o', 'x', 'o', 'b', 'o', 'x', 'o', 'x', 'x'] )
classLabel

In [ ]:
# Step 8
# Get test_dataset from test.csv

test_feature = pd.read_csv('test.csv')
test_dataSet_totalSplit = test_feature.values[:,1:]

def predict(myDecisionTree, featureNames, test_dataset):

    #print test
    myCount = 0
    myClassLabels = []
    ID = []
    for feature in test_dataSet_totalSplit:
        currentClassLabel = classify(myDecisionTree, featureNames, feature)
        print myCount
        print currentClassLabel
    
        myClassLabels.append(currentClassLabel)
        ID.append(myCount)
        myCount +=1
        
    return myClassLabels

In [ ]:
# Step 8 Test:

myPredictions = predict(myDecisionTree, featureNames_copy, test_dataSet_totalSplit)

In [ ]:
# Step 9:
# Output
# mySubmit = pd.read_csv('my_submit.csv')

# 得到最终答案 y_pred, 是一个 1维的array
# 存储为要求格式的文件

df = pd.DataFrame(np.stack( (range(len(myPredictions)), myPredictions) ).T) 
df.to_csv('result.csv', index = None, header=['ID', 'Category'])